# Example 4 - using xcube with a geographic data base

This notebook shows how to access different data sets through the Sentinel Hub API:

1. Query geoDB to retrieve vector data set
2. Rasterize vector data
3. Relate vector data and gridded data retrieved through Sentinel Hub

To run this Notebook, make sure the SENTINEL Hub / xcube Integration is setup correctly, see [Ex0-DCFS-Setup](./Ex0-DCFS-Setup.ipynb).

In [1]:
from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
from xcube_sh.sentinelhub import SentinelHub
import xarray as xr
import numpy as np

In [2]:
x1 = 14.2  # degree
y1 = 44.75  # degree
x2 = 14.75  # degree
y2 = 45.25  # degree

bbox = x1, y1, x2, y2

In [3]:
spatial_res = 0.00018   # = 20.038 meters in degree

In [4]:
SH = SentinelHub()
SH.dataset_names

['S2L1C', 'DEM', 'S2L2A', 'S1GRD', 'CUSTOM']

## Sentinel-2 L2A

In [39]:
SH.band_names('S2L2A')

['B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B8A',
 'B09',
 'B11',
 'B12',
 'SCL',
 'SNW',
 'CLD',
 'viewZenithMean',
 'viewAzimuthMean',
 'sunZenithAngles',
 'sunAzimuthAngles',
 'AOT']

In [13]:
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04'],
                         tile_size=[512, 512],
                         geometry=bbox,
                         spatial_res=spatial_res,
                         time_range=['2019-05-14', '2019-07-31'],
                         time_tolerance='30M')  

In [14]:
cube = open_cube(cube_config)
cube

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 3072, lon: 3072, time: 31)
Coordinates:
  * lat        (lat) float64 45.3 45.3 45.3 45.3 ... 44.75 44.75 44.75 44.75
  * lon        (lon) float64 14.2 14.2 14.2 14.2 ... 14.75 14.75 14.75 14.75
  * time       (time) datetime64[ns] 2019-05-16T10:08:22 ... 2019-07-30T10:08:18
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(31, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    B04        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   S2L2A Data Cube Subset
    history:                 [{'program': 'xcube_sh.store.SentinelHubStore', ...
    date_created:            2019-11-12T23:15:55.816746
    processing_level:        L2A
    time_coverage_start:     2019-05-16T10:08:15+00:00
    time_coverage_end:       2019-07-30T10:08:25+00:00
    time_coverage_duration:  P75DT0H0M10S
    geospatial_lon_m

In [15]:
cube.B04.isel(time=4).plot.imshow(cmap='gray')

In [5]:
from xcube_sh.geodb import get_geo_db_service

In [20]:
geo_db = get_geo_db_service(driver='pgsql', host="db-dcfs-geodb.cbfjgqxk302m.eu-central-1.rds.amazonaws.com",
            user="postgres",
            password="Oeckel6b&z")

In [21]:
geo_db.collections

['geodb_master',
 'geodb_test',
 'geodb_test3',
 'geodb_germany_sh_lakes',
 'land_use',
 'geodb_germany_sh_lakes_long']

In [22]:
fs = geo_db.find_feature(collection_name='germany_sh_lakes', query="'S_NAME' = 'Selenter_See'")

In [24]:
from IPython.display import GeoJSON

In [25]:
GeoJSON(fs)

<IPython.display.GeoJSON object>

In [12]:
fs

{'type': 'Feature',
 'properties': {'id': '0',
  'METADATA_U': 'http://www.wasserblick.net/servlet/is/20090/?title=LWSEGGEOM_DESHnull=104192',
  'RESTRICTED': '1',
  'TEMPLATE': 'Lwseggeom',
  'S_NAME': 'Hemmelmarker_See',
  'EU_CD_LS': 'DE_LS_DESH_0145',
  'EU_CD_LW': 'DE_LW_DESH_0145',
  'LAKE_CAT': '0',
  'RBD_CD': '9610',
  'WA_CD': '9610',
  'PLANU_CD': 'STR_SLE',
  'LAND_CD': 'DESH',
  'INS_WHEN': 'FME Date (2013-11-13',
  'INS_BY': 'G. Baltes',
  'SCALE': 'H',
  'F_MEASURE': 0.0,
  'T_MEASURE': 0.0,
  'INV_LAND': 'DESH',
  'METADATA': 'LWSEGGEOM_DESH.xml',
  'AREA_CALC': 0.818,
  'OID': 110670,
  'IMPORTDATE': 'FME Date (2015-12-14',
  'AREA_MAX': 0.818,
  'CAND_JOIN': 1.0,
  'LIABLE': 'DESH',
  'WHY_HMWB': None,
  'ALT_CAT': None,
  'APSFR_CD': None,
  'BEN_INV': None,
  'C_STAT_DIF': None,
  'CHAR_TYPE': None,
  'CONF_CHEM': None,
  'CONFIDENCE': None,
  'DEPTH_CAT': None,
  'EQS_HM': None,
  'EQS_INDPOL': None,
  'EQS_ONATPL': None,
  'EQS_OTHPL': None,
  'EQS_PESTIC': None,


In [ ]:
keys